# 🧠 Brain Tumor Segmentation with U-Net & Attention U-Net (Showcase Notebook)
**Internship Project | Deep Learning | PyTorch**

---
This notebook demonstrates a full pipeline for brain tumor segmentation using U-Net and Attention U-Net.
Includes full training loop, Dice metric, validation, visualizations, early stopping, learning rate scheduling,
and export options for deployment.


## ⚙️ Setup and Dependencies

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from glob import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('✅ Using device:', DEVICE)

✅ Using device: cuda


In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


CUDA available: True
CUDA device: NVIDIA GeForce RTX 4080 Laptop GPU


In [3]:
# Confirm GPU
print(torch.cuda.get_device_name(0))

# Confirm one batch
for images, masks in train_loader:
    print(images.shape, masks.shape)
    break

# Confirm model forward
model = UNet().to(DEVICE)
print(model(torch.randn(1, 1, 256, 256).to(DEVICE)).shape)


NVIDIA GeForce RTX 4080 Laptop GPU


NameError: name 'train_loader' is not defined

## 📂 Dataset: 4-Channel MRI Loader (.npy format)
Each image is a 4-channel MRI slice saved as `.npy` (shape: HxWx4), and the mask is a 2D array with integer labels.

In [4]:
class BrainTumorDataset(Dataset):
    def __init__(self, image_dir, mask_dir):
        self.image_paths = sorted(glob(os.path.join(image_dir, '*.npy')))
        self.mask_paths = sorted(glob(os.path.join(mask_dir, '*.npy')))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.load(self.image_paths[idx])
        mask = np.load(self.mask_paths[idx])
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        mask = torch.tensor(mask, dtype=torch.long)
        return image, mask

## 🧠 U-Net and Attention U-Net Architectures

In [5]:
class UNet(nn.Module):
    def __init__(self, in_channels=4, out_channels=2):
        super(UNet, self).__init__()
        def conv_block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, padding=1), nn.BatchNorm2d(out_c), nn.ReLU(),
                nn.Conv2d(out_c, out_c, 3, padding=1), nn.BatchNorm2d(out_c), nn.ReLU())
        self.enc1 = conv_block(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.bottleneck = conv_block(128, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = conv_block(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec1 = conv_block(128, 64)
        self.output = nn.Conv2d(64, out_channels, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool1(e1))
        b = self.bottleneck(self.pool2(e2))
        d2 = self.dec2(torch.cat([self.up2(b), e2], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e1], dim=1))
        return self.output(d1)

In [6]:
class AttentionBlock(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionBlock, self).__init__()
        self.W_g = nn.Sequential(nn.Conv2d(F_g, F_int, 1), nn.BatchNorm2d(F_int))
        self.W_x = nn.Sequential(nn.Conv2d(F_l, F_int, 1), nn.BatchNorm2d(F_int))
        self.psi = nn.Sequential(nn.Conv2d(F_int, 1, 1), nn.BatchNorm2d(1), nn.Sigmoid())
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        return x * self.psi(self.relu(self.W_g(g) + self.W_x(x)))

In [7]:
class AttentionUNet(nn.Module):
    def __init__(self, in_channels=4, out_channels=2):
        super(AttentionUNet, self).__init__()
        def conv_block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, padding=1), nn.BatchNorm2d(out_c), nn.ReLU(),
                nn.Conv2d(out_c, out_c, 3, padding=1), nn.BatchNorm2d(out_c), nn.ReLU())
        self.enc1 = conv_block(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.bottleneck = conv_block(128, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.att2 = AttentionBlock(128, 128, 64)
        self.dec2 = conv_block(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.att1 = AttentionBlock(64, 64, 32)
        self.dec1 = conv_block(128, 64)
        self.output = nn.Conv2d(64, out_channels, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool1(e1))
        b = self.bottleneck(self.pool2(e2))
        d2 = self.up2(b); d2 = self.att2(g=d2, x=e2); d2 = self.dec2(torch.cat([d2, e2], dim=1))
        d1 = self.up1(d2); d1 = self.att1(g=d1, x=e1); d1 = self.dec1(torch.cat([d1, e1], dim=1))
        return self.output(d1)

## 🔀 Dataset Split: Train & Validation
We use an 80/20 split to train and evaluate performance.

In [8]:
dataset = BrainTumorDataset(
    r"D:\\Khedir-meriem-ESI-SBEIAbes\\data\\input_data_4channels_z_score\\train\\images",
    r"D:\\Khedir-meriem-ESI-SBEIAbes\\data\\input_data_4channels_z_score\\train\\masks"
)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_set, batch_size=4, shuffle=True)
val_loader = DataLoader(val_set, batch_size=4, shuffle=False)
print(f"Train: {len(train_set)} | Validation: {len(val_set)}")

ValueError: num_samples should be a positive integer value, but got num_samples=0

## 📐 Dice Coefficient Metric
Used to evaluate how well the predicted segmentation matches the ground truth.

In [ ]:
def dice_coeff(pred, target, epsilon=1e-6):
    pred = pred.contiguous().view(-1)
    target = target.contiguous().view(-1)
    intersection = (pred * target).sum()
    return (2. * intersection + epsilon) / (pred.sum() + target.sum() + epsilon)

## 🧪 Training Function (AMP + EarlyStopping + Scheduler)

In [ ]:
from torch.cuda.amp import autocast, GradScaler

def train_model(model, name):
    model = model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)
    criterion = nn.CrossEntropyLoss()

    scaler = GradScaler()
    best_dice = 0
    patience = 10
    trigger = 0
    train_dice, val_dice = [], []

    for epoch in range(1, 101):
        model.train()
        total_loss = 0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch}"):
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()

            with autocast():  # Mixed precision training
                out = model(x)
                loss = criterion(out, y)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

        # Validation
        model.eval()
        dice = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                with autocast():  # Use AMP for inference too
                    pred = torch.argmax(model(x), dim=1)
                dice += dice_coeff(pred.cpu(), y.cpu())
        avg_dice = dice / len(val_loader)
        val_dice.append(avg_dice.item())

        print(f"[{name}] Epoch {epoch}, Loss: {total_loss:.4f}, Val Dice: {avg_dice:.4f}")
        scheduler.step(total_loss)

        if avg_dice > best_dice:
            best_dice = avg_dice
            torch.save(model.state_dict(), f"{name}_best.pth")
            print(f"✅ Saved best model with Dice: {best_dice:.4f}")
            trigger = 0
        else:
            trigger += 1
            if trigger >= patience:
                print("⏹️ Early stopping triggered.")
                break


## 🧠 Train Both Models (100 Epochs Max + Early Stopping)

In [ ]:
train_model(UNet(), "unet")
train_model(AttentionUNet(), "attention_unet")

## 📊 Plot Dice Score Over Epochs

In [ ]:
# If you store dice per epoch above, you can plot:
# plt.plot(train_dice, label='Train Dice')
# plt.plot(val_dice, label='Val Dice')
# plt.legend(); plt.xlabel('Epoch'); plt.ylabel('Dice'); plt.title('Training Curve'); plt.show()

## 🔍 Visualize Predictions

In [ ]:
def visualize_predictions(model_path, model_class):
    model = model_class().to(DEVICE)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    x, y = next(iter(val_loader))
    x, y = x.to(DEVICE), y.to(DEVICE)
    with torch.no_grad():
        pred = torch.argmax(model(x), dim=1)

    plt.figure(figsize=(12, 4))
    for i in range(3):
        plt.subplot(3, 3, i*3+1); plt.imshow(x[i][0].cpu(), cmap='gray'); plt.title('Input')
        plt.subplot(3, 3, i*3+2); plt.imshow(y[i].cpu(), cmap='gray'); plt.title('GT')
        plt.subplot(3, 3, i*3+3); plt.imshow(pred[i].cpu(), cmap='gray'); plt.title('Prediction')
    plt.tight_layout(); plt.show()

visualize_predictions("unet_best.pth", UNet)
visualize_predictions("attention_unet_best.pth", AttentionUNet)

## 💾 Export Models: TorchScript + ONNX

In [ ]:
dummy = torch.randn(1, 4, 256, 256).to(DEVICE)
model = AttentionUNet().to(DEVICE)
model.load_state_dict(torch.load("attention_unet_best.pth"))
torch.jit.trace(model, dummy).save("attention_unet.pt")
torch.onnx.export(model, dummy, "attention_unet.onnx", input_names=["input"], output_names=["output"], opset_version=11)
print("✅ Exported to .pt and .onnx")

## ✅ Summary & Deployment Tips
- Best Dice model saved as `.pth`
- Deployment-ready formats: `.pt` (TorchScript), `.onnx`
- Use ONNX for ONNX Runtime, TensorRT, or OpenVINO.
- Ideal for mobile, embedded, or web deployment.

**Next step:** Try converting the ONNX model into TensorRT or integrating in a simple Flask demo.

## 🧪 Advanced Data Augmentation with Albumentations
We use Albumentations to apply more realistic and effective augmentations during training.

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

class AugmentedBrainTumorDataset(Dataset):
    def __init__(self, image_dir, mask_dir, augment=False):
        self.image_paths = sorted(glob(os.path.join(image_dir, '*.npy')))
        self.mask_paths = sorted(glob(os.path.join(mask_dir, '*.npy')))
        self.augment = augment
        self.transform = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Rotate(limit=15, p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Normalize(),
            ToTensorV2()
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.load(self.image_paths[idx]).astype(np.float32)
        mask = np.load(self.mask_paths[idx]).astype(np.uint8)
        image = image.transpose(2, 0, 1)  # [C, H, W]
        if self.augment:
            augmented = self.transform(image=image.transpose(1, 2, 0), mask=mask)
            image, mask = augmented['image'], augmented['mask']
        else:
            image = torch.tensor(image, dtype=torch.float32)
            mask = torch.tensor(mask, dtype=torch.long)
        return image, mask

In [ ]:
# Replace old dataset and reload
dataset = AugmentedBrainTumorDataset(
    r"D:\\Khedir-meriem-ESI-SBEIAbes\\data\\input_data_4channels_z_score\\train\\images",
    r"D:\\Khedir-meriem-ESI-SBEIAbes\\data\\input_data_4channels_z_score\\train\\masks",
    augment=True
)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_set, batch_size=4, shuffle=True)
val_loader = DataLoader(val_set, batch_size=4, shuffle=False)

## 🔗 Ensemble Prediction: U-Net + Attention U-Net
Combines predictions from both models to potentially improve segmentation accuracy.

In [ ]:
def ensemble_predict(x):
    model1 = UNet().to(DEVICE)
    model1.load_state_dict(torch.load("unet_best.pth"))
    model2 = AttentionUNet().to(DEVICE)
    model2.load_state_dict(torch.load("attention_unet_best.pth"))
    model1.eval(); model2.eval()

    with torch.no_grad():
        out1 = model1(x)
        out2 = model2(x)
        avg_out = (out1 + out2) / 2
        pred = torch.argmax(avg_out, dim=1)
    return pred

In [ ]:
# Visualize ensemble prediction
x, y = next(iter(val_loader))
x, y = x.to(DEVICE), y.to(DEVICE)
pred = ensemble_predict(x)

plt.figure(figsize=(12, 4))
for i in range(3):
    plt.subplot(3, 3, i*3+1); plt.imshow(x[i][0].cpu(), cmap='gray'); plt.title('Input')
    plt.subplot(3, 3, i*3+2); plt.imshow(y[i].cpu(), cmap='gray'); plt.title('Ground Truth')
    plt.subplot(3, 3, i*3+3); plt.imshow(pred[i].cpu(), cmap='gray'); plt.title('Ensemble Pred')
plt.tight_layout(); plt.show()